In [ ]:
import os
import pandas as pd
import matplotlib as mpl
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf
import math
from tensorflow import keras
from tensorflow.keras import layers
from tensorflow.keras import Sequential
from keras.layers import Bidirectional
from tensorflow.keras.layers import LSTM
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import LeakyReLU
from keras.callbacks import EarlyStopping
from keras.layers import Bidirectional
from statsmodels.tsa.api import VAR
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import train_test_split
import rpy2.robjects as robjects # r 함수를 파이썬에서 사용 가능하게 변환하는 모듈
from rpy2.robjects import pandas2ri # 파이썬 자료형과 R 자료형의 호환을 도와주는 모듈
from tqdm import tqdm

/usr/local/lib/python3.7/dist-packages/statsmodels/tools/_testing.py:19: FutureWarning: pandas.util.testing is deprecated. Use the functions in the public API at pandas.testing instead.
  import pandas.util.testing as tm


In [ ]:
import warnings
warnings.filterwarnings(action='ignore')

In [ ]:
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3' 

## Util

In [ ]:
def create_lstm_dataset(dataset, look_back=1): #LSTM
    dataX, dataY = [], []
    for i in range(len(dataset)-look_back-1):
        a = dataset[i:(i+look_back), 0]
        dataX.append(a)
        dataY.append(dataset[i + look_back, 0])
    return np.array(dataX), np.array(dataY)

In [ ]:
def coefficient_variation(df, i): #ETS
    cv_data = df.groupby(['emd_nm']).em_g.std()/df.groupby(['emd_nm']).em_g.mean()
    cv = cv_data[i]
    return cv

In [ ]:
def month_resampling(df): #ETS
    new_data = pd.DataFrame() 
    df['base_date'] = df['base_date'].astype(str)
    df['year_month'] = df['base_date'].str.slice(stop=7)
    year_month = df['year_month'].drop_duplicates()

    downsampling_data = df.groupby(['emd_nm', 'year_month']).em_g.sum()
    downsampling_data = pd.DataFrame(downsampling_data)
    downsampling_data = downsampling_data.reset_index(drop=False,inplace=False)
    
    for i in tqdm(df.emd_nm.unique()):
        jiyeok = downsampling_data[downsampling_data['emd_nm']==i]

        jiyeok = jiyeok.merge(year_month,how='outer')

        jiyeok = jiyeok.sort_values(by=['year_month'], axis=0, ascending=True) 

        jiyeok['emd_nm'] = jiyeok['emd_nm'].fillna(i)
        
        new_data = pd.concat([new_data,jiyeok],axis=0)
        
    return new_data

## Data

In [ ]:
r_df = pd.read_csv('final4.csv', encoding='euc-kr')
r_df = r_df.drop(['Unnamed: 0'], axis=1) #raw data
df = r_df.dropna()
make_group = df[['em_g','resid_reg_pop','foreign_pop','total_pop','외식_use_amt','장보기_use_amt','배달_use_amt', 'em_cnt']].groupby([df['base_year'],df['base_month'],df['emd_nm']])
df2 = pd.DataFrame(make_group.sum()).reset_index() #grouped data

r_df2 = r_df[r_df['emd_nm']=='알수없음']
make_group2 = r_df2[['em_g']].groupby([r_df2['base_year'],r_df2['base_month'],r_df2['emd_nm']])
x_df = pd.DataFrame(make_group2.sum()).reset_index().drop(['emd_nm', 'base_year', 'base_month'], axis=1) #알수없음 data

In [ ]:
group_0 = ['도두동', '오라동', '건입동', '봉개동', '삼도1동', '이도1동', '용담2동', '이호동',
           '동홍동', '대정읍', '영천동', '서홍동', '효돈동', '중문동', '대륜동', '남원읍', '대천동', '안덕면', '예래동', '표선면', '성산읍', '정방동', '송산동', '천지동', '중앙동', '삼도2동', '일도1동', '용담1동', '구좌읍', '한경면']
group_1 = ['노형동', '이도2동', '연동']
group_2 = ['외도동', '아라동', '삼양동', '일도2동', '화북동', '애월읍', '조천읍', '한림읍']

## LSTM

In [ ]:
def predict_BiLSTM(emd_nm):
    emd_nm = emd_nm
    
    if emd_nm == '알수없음':
      df3 = x_df

    else:
      df3 = df2[df2['emd_nm']==emd_nm]
      df3 = df3.drop(columns=['resid_reg_pop', 'resid_reg_pop', 'foreign_pop', 'total_pop',
                     '외식_use_amt', '장보기_use_amt', '배달_use_amt', 'em_cnt', 'emd_nm',
                       'base_year', 'base_month'])
    
    data = df3.values
    dataset = data.astype('float32')
    
    scaler = MinMaxScaler(feature_range=(0, 1))
    dataset = scaler.fit_transform(dataset)

    train_size = int(len(dataset)-4)

    test_size = len(dataset) - train_size
    train, test = dataset[0:train_size,:], dataset[train_size:len(dataset),:]

    
    look_back = 1
    trainX, trainY = create_lstm_dataset(train, look_back)
    testX, testY = create_lstm_dataset(test, look_back)
    
    trainX = np.reshape(trainX, (trainX.shape[0], 1, trainX.shape[1]))
    testX = np.reshape(testX, (testX.shape[0], 1, testX.shape[1]))
    
    model = Sequential()
    model.add(Bidirectional(LSTM(4, activation='relu'), input_shape=(1, look_back)))
    model.add(Dense(1))
    model.compile(optimizer='adam', loss='mse')

    early_stop = EarlyStopping(monitor='loss', patience=3, verbose=1)

    model.fit(trainX, trainY, epochs=200, batch_size=1, verbose=0, callbacks=[early_stop])
    
    #july prediction
    look_back=1
    predict_july = np.reshape(testY, (testY.shape[0], 1, 1))
    prediction = model.predict(predict_july)
    july = scaler.inverse_transform(prediction)[0][0]
    
    #august prediction
    input_2 = []
    input_2.append(testY[1])
    input_2.append(july)
    input_2 = np.array(input_2)
    
    predict_august = np.reshape(input_2, (input_2.shape[0], 1, 1))
    prediction_2 = model.predict(predict_august)
    august = scaler.inverse_transform(prediction_2)[0][0]
    
    
    return july, august

## Random Forest

In [ ]:
def RandomForest(group):
    emd_list = group
    df3 = df2[df2['emd_nm'].isin(emd_list)]
    
    features = df3
    features = pd.get_dummies(features)
    
    labels = np.array(features['em_g'])
    features = features.drop('em_g', axis=1)
    feature_list = list(features.columns)
    features = np.array(features)
    
    train_features, test_features, train_labels, test_labels = train_test_split(features, labels, test_size = 0.25, random_state = 42)
    
    rf = RandomForestRegressor(n_estimators = 1000, random_state = 42)
    rf.fit(train_features, train_labels)
    
    input_j=df3.groupby('emd_nm').mean().reset_index().drop(['em_g'], axis=1)
    input_j['base_year']=2021
    input_j['base_month']=7
    input_data_july = pd.get_dummies(input_j)
    
    july_predicted_emg = []

    for i in range(len(input_data_july)):
        input_values = input_data_july.iloc[i].values.reshape((-1, train_features.shape[1]))
        predicted_value = rf.predict(input_values)[0]
        july_predicted_emg.append(predicted_value)
    
    
    input_a=df3.groupby('emd_nm').mean().reset_index().drop(['em_g'], axis=1)
    input_a['base_year']=2021
    input_a['base_month']=8
    input_data_august = pd.get_dummies(input_a)
    
    august_predicted_emg = []

    for i in range(len(input_data_august)):
        input_values = input_data_august.iloc[i].values.reshape((-1, train_features.shape[1]))
        predicted_value = rf.predict(input_values)[0]
        august_predicted_emg.append(predicted_value)
    
    input_j['7월_predicted_emg'] = july_predicted_emg
    input_j['8월_predicted_emg'] = august_predicted_emg
    
    result_data = input_j.drop(['resid_reg_pop','foreign_pop','total_pop','외식_use_amt','장보기_use_amt','배달_use_amt', 'em_cnt', 'base_year', 'base_month'], axis=1)
    

    return result_data

## VAR

In [ ]:
def predict_VAR(emd_nm):
    
    make_group = r_df[['em_g','resid_reg_pop','foreign_pop','total_pop','외식_use_amt','장보기_use_amt','배달_use_amt', 'em_cnt']].groupby([r_df['base_date'],r_df['base_year'],r_df['base_month'],r_df['emd_nm']])
    df = pd.DataFrame(make_group.sum()).reset_index()
    df['base_date']= pd.to_datetime(df['base_date'])
    df = df.set_index('base_date')
    df = df.drop(['base_year', 'base_month'], axis=1)
    
    df2 = df[df['emd_nm']==emd_nm]
    VARdf =df2.resample('1M').sum()

    if emd_nm in group_0:
        df_input=VARdf[['em_g','외식_use_amt', '장보기_use_amt']]
    if emd_nm in group_1:
        df_input=VARdf[['em_g','total_pop', '외식_use_amt', '장보기_use_amt']]
    if emd_nm in group_2:
        df_input=VARdf[['em_g','resid_reg_pop', 'total_pop', '외식_use_amt']]

    df_train = df_input[(df_input.index>='2018-01-01') & (df_input.index<='2021-04-30')]
    df_test = df_input[(df_input.index>='2021-05-01') & (df_input.index<='2021-06-30')]
    
    if emd_nm in group_0:
        lag_val = 2
    if emd_nm in group_1:
        lag_val = 1
    if emd_nm in group_2:
        lag_val = 3
    
    model = VAR(df_train, freq="1M")
    results = model.fit(lag_val)
    
    lag=results.k_ar
    
    forecasted_results = results.forecast(df_input.values[-lag:],steps=2)
    
    forecasted_emg = []
    for i in range(len(forecasted_results)):
        emg = forecasted_results[i][0]
        forecasted_emg.append(emg)
    

    return forecasted_emg

## ETS

In [ ]:
from rpy2.robjects.packages import importr

utils = importr('utils')
utils.install_packages('forecast')
utils.install_packages('forecastHybrid')

R[write to console]: Installing package into ‘/usr/local/lib/R/site-library’
(as ‘lib’ is unspecified)

R[write to console]: also installing the dependencies ‘xts’, ‘TTR’, ‘quadprog’, ‘quantmod’, ‘fracdiff’, ‘lmtest’, ‘timeDate’, ‘tseries’, ‘urca’, ‘zoo’, ‘RcppArmadillo’


R[write to console]: trying URL 'https://cran.rstudio.com/src/contrib/xts_0.12.1.tar.gz'

R[write to console]: Content type 'application/x-gzip'
R[write to console]:  length 517875 bytes (505 KB)

R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =
R[write to console]: =


<rpy2.rinterface_lib.sexp.NULLType object at 0x7f9ec0f2aaf0> [RTYPES.NILSXP]

In [ ]:
# pandas2ri를 활성화 
pandas2ri.activate()

In [ ]:
ets = """
    function(ts){
        library(forecast) # forecast 패키지 로드
        model = ets(ts) # AIC가 낮은 지수평활 모델을 찾음 
        forecasted_data = forecast(model, h=2) # 이후 2개월(h=2)을 예측
        out_df = data.frame(forecasted_data$mean) # 예측값을 R의 데이터프레임으로 변환
        colnames(out_df) = c('amount') # amount라는 열로 이름을 지정
        out_df
    }
"""

# r() 함수로 r 자료형을 파이썬에서 사용 가능
ets = robjects.r(ets)
ts = robjects.r('ts') # r 자료형 time series 자료형으로 만들어주는 함수
c = robjects.r('c') # r 자료형 벡터를 만들어주는 함수
log = robjects.r('log') # 로그 변환 함수
exp = robjects.r('exp')# 로그 역변환 함수

In [ ]:
def predict_ETS(group):
    emd_list = group

    final_pred_1 = []
    final_pred_2 = []
    dong = []

    if group == '알수없음':
      train = r_df[r_df['emd_nm']==emd_list]
      train['base_date'] = pd.to_datetime(train['base_date'],format="%Y-%m-%d")
    
    else:
      train = r_df[r_df['emd_nm'].isin(emd_list)]
      train['base_date'] = pd.to_datetime(train['base_date'],format="%Y-%m-%d")

    resampling_data = month_resampling(train)

    for i in tqdm(resampling_data.emd_nm.unique()):
      trash = resampling_data[resampling_data['emd_nm']==i]

      cv = coefficient_variation(resampling_data, i)
      # 매출액 변동 계수가 0.3 미만인 경우만 log를 씌움
      if cv < 0.3:
        train_log = ts(log(trash['em_g']), frequency=12) 
        # ets model
        forecast_log = ets(train_log)
        final_pred_1.append(float(exp(forecast_log.values[0])))
        final_pred_2.append(float(exp(forecast_log.values[1])))
        dong.append(i)
        # 매출액 변동 계수가 0.3 이상인 경우
      else:
        train = ts(trash['em_g'], frequency=12)
        # 지수평활법
        forecast = ets(train)
        final_pred_1.append(float(forecast.values[0]))
        final_pred_2.append(float(forecast.values[1]))
        dong.append(i)
      
      dict_data5_3= {'emd_nm':dong, '7월_predicted_emg':final_pred_1, '8월_predicted_emg': final_pred_2}
      df5_3 = pd.DataFrame(dict_data5_3)
    
    return df5_3

## Ensemble

In [ ]:
def Ensemble(emd_nm):
    
    if emd_nm in group_0:
        r = RandomForest(group_0)
        e = predict_ETS(group_0)
        model2_july_predict = r.loc[r.emd_nm==emd_nm].values[0][1]
        model2_august_predict = r.loc[r.emd_nm==emd_nm].values[0][2]
        model4_july_predict = e.loc[e.emd_nm==emd_nm].values[0][1]
        model4_august_predict = e.loc[e.emd_nm==emd_nm].values[0][2]
        model1_july_predict = predict_BiLSTM(emd_nm)[0]    
        model1_august_predict = predict_BiLSTM(emd_nm)[1]

        weights = np.array([0.82, 0.70, 0.71])

        # weighted average
        july_predictions = np.array([model1_july_predict, model2_july_predict, model4_july_predict])
        august_predictions = np.array([model1_august_predict, model2_august_predict, model4_august_predict])

        final_preds_july = july_predictions * weights
        final_preds_august = august_predictions * weights

        final_pred_july = np.sum(final_preds_july) / np.sum(weights)
        final_pred_august = np.sum(final_preds_august) / np.sum(weights)

    elif emd_nm in group_1:
        r = RandomForest(group_1)
        e = predict_ETS(group_1)
        model2_july_predict = r.loc[r.emd_nm==emd_nm].values[0][1]
        model2_august_predict = r.loc[r.emd_nm==emd_nm].values[0][2]
        model4_july_predict = e.loc[e.emd_nm==emd_nm].values[0][1]
        model4_august_predict = e.loc[e.emd_nm==emd_nm].values[0][2]

        weights = np.array([0.97, 0.97, 0.99, 0.89])

        model1_july_predict = predict_BiLSTM(emd_nm)[0]    
        model1_august_predict = predict_BiLSTM(emd_nm)[1]
        model3_july_predict = predict_VAR(emd_nm)[0] 
        model3_august_predict = predict_VAR(emd_nm)[1]

        # weighted average
        july_predictions = np.array([model1_july_predict, model2_july_predict, model3_july_predict, model4_july_predict])
        august_predictions = np.array([model1_august_predict, model2_august_predict, model3_august_predict, model4_august_predict])

        final_preds_july = july_predictions * weights
        final_preds_august = august_predictions * weights
        
        final_pred_july = np.sum(final_preds_july) / np.sum(weights)
        final_pred_august = np.sum(final_preds_august) / np.sum(weights)

    elif emd_nm in group_2:
        r = RandomForest(group_2)
        e = predict_ETS(group_2)
        model2_july_predict = r.loc[r.emd_nm==emd_nm].values[0][1]
        model2_august_predict = r.loc[r.emd_nm==emd_nm].values[0][2]
        model4_july_predict = e.loc[e.emd_nm==emd_nm].values[0][1]
        model4_august_predict = e.loc[e.emd_nm==emd_nm].values[0][2]
        weights = np.array([0.89, 0.81, 0.85, 0.81])
        
        model1_july_predict = predict_BiLSTM(emd_nm)[0]    
        model1_august_predict = predict_BiLSTM(emd_nm)[1]
        model3_july_predict = predict_VAR(emd_nm)[0] 
        model3_august_predict = predict_VAR(emd_nm)[1]

        # weighted average
        july_predictions = np.array([model1_july_predict, model2_july_predict, model3_july_predict, model4_july_predict])
        august_predictions = np.array([model1_august_predict, model2_august_predict, model3_august_predict, model4_august_predict])

        final_preds_july = july_predictions * weights
        final_preds_august = august_predictions * weights
        
        final_pred_july = np.sum(final_preds_july) / np.sum(weights)
        final_pred_august = np.sum(final_preds_august) / np.sum(weights)
    
    elif emd_nm=='알수없음':
      model1_july_predict = predict_BiLSTM('알수없음')[0]
      model1_august_predict = predict_BiLSTM('알수없음')[1]

      model4_july_predict = predict_ETS('알수없음').values[0][1]
      model4_august_predict = predict_ETS('알수없음').values[0][2]

      weights = np.array([0.4378, 0.5621])

      # weighted average
      july_predictions = [model1_july_predict, model4_july_predict]
      august_predictions = [model1_august_predict, model4_august_predict]

      final_preds_july = july_predictions * weights
      final_preds_august = august_predictions * weights
          
      final_pred_july = np.sum(final_preds_july) / np.sum(weights)
      final_pred_august = np.sum(final_preds_august) / np.sum(weights)
      
    return int(final_pred_july), int(final_pred_august)
    

## Prediction

In [ ]:
emdnm_list = ['한림읍', '애월읍', '구좌읍', '조천읍', '한경면', '일도1동', '일도2동', '이도1동',
              '이도2동', '삼도1동', '삼도2동', '용담1동', '용담2동', '건입동', '화북동',
              '삼양동', '봉개동', '아라동', '오라동', '연동', '노형동', '외도동', '이호동',
              '도두동', '대정읍', '남원읍', '성산읍', '안덕면', '표선면', '송산동', '정방동',
              '중앙동', '천지동', '효돈동', '영천동', '동홍동', '서홍동', '대륜동', '대천동',
              '중문동', '예래동', '알수없음']

predictions2 = []
for i in range(len(emdnm_list)):
  predictions2.append(Ensemble(emdnm_list[i]))
  print(emdnm_list[i], '완료 (%d/42)' %(i+1))

final2 = pd.DataFrame(predictions2, columns=['7월 배출량 (g)', '8월 배출량 (g)'], index = emdnm_list)
final2

100%|██████████| 8/8 [00:00<00:00, 31.90it/s]


Epoch 00036: early stopping
Epoch 00046: early stopping
한림읍 완료 (1/42)


100%|██████████| 8/8 [00:00<00:00, 33.17it/s]


Epoch 00053: early stopping
Epoch 00042: early stopping
애월읍 완료 (2/42)


100%|██████████| 30/30 [00:00<00:00, 36.49it/s]


...

100%|██████████| 30/30 [00:00<00:00, 37.69it/s]


Epoch 00044: early stopping
Epoch 00084: early stopping
예래동 완료 (41/42)
Epoch 00127: early stopping
Epoch 00075: early stopping


100%|██████████| 1/1 [00:00<00:00, 47.86it/s]

알수없음 완료 (42/42)


,7월 배출량 (g),8월 배출량 (g)
한림읍,112334033,103389830
애월읍,161561177,164617447
구좌읍,41829855,41974501
조천읍,87993305,80613070
한경면,26107581,22712446
일도1동,24118682,24441494
일도2동,251012206,245360142
이도1동,62887662,64334602
이도2동,360001661,362441699
삼도1동,104744844,107584740
